In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import os


def set_environment_variables(env_path: str = '../.env',):
    with open(env_path, 'r') as f:
        key, val = f.readline().split()
        os.environ[key] = val

In [4]:
set_environment_variables()

In [5]:
import os, sys

print(os.environ.get("SHIT"))

None


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter 
from langchain.document_loaders import TextLoader

loader = TextLoader('../preprocessedDocs/skillfactory.txt')
# loader = TextLoader('../burnout.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
db

/tmp/ipykernel_11362/2772616116.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/sergey/Projects/BurnoutAnalysis/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke("Я хочу питсы")

retrieved_docs[0].page_content

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Честные истории о карьере в IT\nИз-за чего выгорают IT-специалисты и как с этим бороться Личный опыт и советы экспертов\n1 марта 2024\nЭто рубрика в которой эксперты отвечают на актуальные и волнующие вопросы об IT-профессиях будущем сферы и ее перспективах\nКонечно разработчики могут выгорать профессионально Программирование  это тяжелая умственная работа Достаточно сложно писать код и сохранять концентрацию в течение нескольких часов подряд\nКроме того часто программисты особенно backend-разработчики не видят визуальный результат своего труда Это сложная но при этом рутинная работа изо дня в день Разработчики могут выгорать потому что они банально устают от работы\nНаш лучший курс для старта в IT За 2 месяца вы пробуете себя в девяти разных профессиях мобильной и веб-разработке тестировании аналитике и даже Data Science  выберите подходящую и сразу освойте ее\nПопробуйте 9 профессий за 2 месяца и выберите подходящую вам\n4 490 мес\nОпасность выгорания отдельного сотрудника для бизне

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [9]:
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_KEY'],
    model="GigaChat",
    verify_ssl_certs=False,
    profanity_check=False,
)

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)
prompt = ChatPromptTemplate.from_template('''Ну такое
Контекст: {context}
Вопрос: {input} 
Ответ:''')

In [10]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [11]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [12]:
retrieval_chain.invoke({'input': "Можешь рассказать про выгорание в IT-сфере?"})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Giga generation stopped with reason: blacklist


{'input': 'Можешь рассказать про выгорание в IT-сфере?',
 'context': [Document(metadata={'source': '../preprocessedDocs/skillfactory.txt'}, page_content='Честные истории о карьере в IT\nИз-за чего выгорают IT-специалисты и как с этим бороться Личный опыт и советы экспертов\n1 марта 2024\nЭто рубрика в которой эксперты отвечают на актуальные и волнующие вопросы об IT-профессиях будущем сферы и ее перспективах\nКонечно разработчики могут выгорать профессионально Программирование  это тяжелая умственная работа Достаточно сложно писать код и сохранять концентрацию в течение нескольких часов подряд\nКроме того часто программисты особенно backend-разработчики не видят визуальный результат своего труда Это сложная но при этом рутинная работа изо дня в день Разработчики могут выгорать потому что они банально устают от работы\nНаш лучший курс для старта в IT За 2 месяца вы пробуете себя в девяти разных профессиях мобильной и веб-разработке тестировании аналитике и даже Data Science  выберите по

In [13]:
len(docs[0].page_content)

11955

In [14]:
len(documents)

1

In [15]:
retriever.invoke("Я хочу питсы")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(metadata={'source': '../preprocessedDocs/skillfactory.txt'}, page_content='Честные истории о карьере в IT\nИз-за чего выгорают IT-специалисты и как с этим бороться Личный опыт и советы экспертов\n1 марта 2024\nЭто рубрика в которой эксперты отвечают на актуальные и волнующие вопросы об IT-профессиях будущем сферы и ее перспективах\nКонечно разработчики могут выгорать профессионально Программирование  это тяжелая умственная работа Достаточно сложно писать код и сохранять концентрацию в течение нескольких часов подряд\nКроме того часто программисты особенно backend-разработчики не видят визуальный результат своего труда Это сложная но при этом рутинная работа изо дня в день Разработчики могут выгорать потому что они банально устают от работы\nНаш лучший курс для старта в IT За 2 месяца вы пробуете себя в девяти разных профессиях мобильной и веб-разработке тестировании аналитике и даже Data Science  выберите подходящую и сразу освойте ее\nПопробуйте 9 профессий за 2 месяца и выб

bm25_chain

In [16]:
import string


def tokenize(s):
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")


In [17]:
from langchain.chains import RetrievalQA

embedding_retriever = retriever
bm25_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=tokenize,
    k=3,
)

bm25_chain = create_retrieval_chain(bm25_retriever, document_chain)

In [18]:
bm25_chain.invoke({'input': "Что ты моешь сказать про автора?"})

Giga generation stopped with reason: blacklist


{'input': 'Что ты моешь сказать про автора?',
 'context': [Document(metadata={'source': '../preprocessedDocs/skillfactory.txt'}, page_content='Честные истории о карьере в IT\nИз-за чего выгорают IT-специалисты и как с этим бороться Личный опыт и советы экспертов\n1 марта 2024\nЭто рубрика в которой эксперты отвечают на актуальные и волнующие вопросы об IT-профессиях будущем сферы и ее перспективах\nКонечно разработчики могут выгорать профессионально Программирование  это тяжелая умственная работа Достаточно сложно писать код и сохранять концентрацию в течение нескольких часов подряд\nКроме того часто программисты особенно backend-разработчики не видят визуальный результат своего труда Это сложная но при этом рутинная работа изо дня в день Разработчики могут выгорать потому что они банально устают от работы\nНаш лучший курс для старта в IT За 2 месяца вы пробуете себя в девяти разных профессиях мобильной и веб-разработке тестировании аналитике и даже Data Science  выберите подходящую и 